# Intro:
We're going to see what a winning team looks like statistically with:
- Wins vs losses
- Goals scored vs mediocre team goals scored
- The team attributes

## Import Dependencies

In [280]:
%matplotlib inline
import pandas as pd 
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import matplotlib.ticker as ticker
import seaborn as sns
import sqlite3
import datetime
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Import Data

In [281]:
with sqlite3.connect('database.sqlite') as con:
    countries = pd.read_sql_query("SELECT * from Country", con)
    matches = pd.read_sql_query("SELECT * from Match", con)
    leagues = pd.read_sql_query("SELECT * from League", con)
    teams = pd.read_sql_query("SELECT * from Team", con)
    player = pd.read_sql_query("SELECT * from Player",con)
    player_attributes = pd.read_sql_query("SELECT * from Player_Attributes",con)
    sequence = pd.read_sql_query("SELECT * from sqlite_sequence",con)
    team_attributes = pd.read_sql_query("SELECT * from Team_Attributes",con)

## Merge Data

### Team and Attributes

In [282]:
teamsDF = teams.merge(team_attributes, on=['team_api_id', 'team_fifa_api_id'])
teamsDF.drop(['id_x', 'id_y'], axis=1, inplace=True)
teamsDF.drop_duplicates(subset=['date', 'team_long_name'], inplace = True)
teamsDF.duplicated(subset = ['date', 'team_long_name']).sum()
teamsDF.sample(3)

,team_api_id,team_fifa_api_id,team_long_name,team_short_name,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,chanceCreationCrossing,chanceCreationCrossingClass,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
1114,2033,111540.0,S.C. Olhanense,OLH,2011-02-22 00:00:00,65,Balanced,NaN,Little,55,Mixed,Organised,50,Normal,50,Normal,65,Normal,Free Form,50,Medium,45,Press,35,Normal,Cover
462,9837,379.0,Stade de Reims,REI,2014-09-19 00:00:00,48,Balanced,58.0,Normal,48,Mixed,Organised,48,Normal,52,Normal,50,Normal,Organised,33,Deep,44,Press,50,Normal,Cover
107,10261,13.0,Newcastle United,NEW,2014-09-19 00:00:00,54,Balanced,33.0,Little,47,Mixed,Organised,47,Normal,51,Normal,39,Normal,Organised,56,Medium,54,Press,51,Normal,Cover


In [283]:
teamsDF.shape

(1450, 26)

In [284]:
a, b, c = teams.shape, team_attributes.shape, teamsDF.shape
print('Original "team" shape:            ' + str(a))
print('Original "team attributes" shape: ' + str(b))
print('Modified "teamDF" shape:          ' + str(c))

Original "team" shape:            (299, 5)
Original "team attributes" shape: (1458, 25)
Modified "teamDF" shape:          (1450, 26)


### Country, League, and Match

In [285]:
leaguesDF = countries.merge(leagues, on=['id'])
leaguesDF = leaguesDF.rename(columns={'name_x':'Country', 'name_y':'League'})
leaguesDF.sample(3)

,id,Country,country_id,League
4,10257,Italy,10257,Italy Serie A
9,21518,Spain,21518,Spain LIGA BBVA
0,1,Belgium,1,Belgium Jupiler League


In [286]:
leaguesDF = leaguesDF.merge(matches, on = ['country_id'])
leaguesDF.sample(3)

,id_x,Country,country_id,League,id_y,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
10833,10257,Italy,10257,Italy Serie A,10834,10257,2009/2010,27,2010-03-07 00:00:00,705292,9857,9875,2,1,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,5.0,5.0,1.0,3.0,5.0,7.0,2.0,4.0,6.0,8.0,4.0,6.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,9.0,11.0,1.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,9.0,9.0,11.0,41671.0,24493.0,24792.0,114769.0,30285.0,24464.0,39426.0,25595.0,27680.0,41749.0,24624.0,32746.0,42388.0,32769.0,39569.0,39535.0,56585.0,24546.0,39509.0,18925.0,41658.0,39540.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>43</comment><event...,2.9,3.1,2.5,2.75,3.05,2.55,2.80,3.1,2.5,2.60,3.0,2.5,NaN,NaN,NaN,2.88,3.00,2.4,3.2,3.0,2.38,2.88,3.0,2.5,2.80,3.1,2.50,2.88,3.1,2.4
18049,17642,Portugal,17642,Portugal Liga ZON Sagres,18050,17642,2009/2010,29,2010-05-02 00:00:00,686314,7841,7844,0,0,1.0,2.0,4.0,6.0,8.0,3.0,5.0,7.0,3.0,5.0,7.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,10.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,40841.0,11860.0,11349.0,150313.0,150081.0,150083.0,150082.0,43295.0,43294.0,25929.0,164367.0,97363.0,97493.0,42463.0,45285.0,46386.0,11897.0,121960.0,28461.0,25998.0,97488.0,149845.0,None,None,None,None,None,None,None,None,3.5,3.1,2.2,3.30,3.00,2.15,3.10,3.0,2.2,3.00,3.2,2.1,NaN,NaN,NaN,3.00,3.40,2.0,3.5,3.2,2.00,3.40,3.0,2.1,3.10,3.1,2.20,3.10,3.0,2.2
13364,13274,Netherlands,13274,Netherlands Eredivisie,13365,13274,2008/2009,19,2009-01-24 00:00:00,523813,10228,9803,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.5,4.2,6.0,1.45,3.90,6.00,1.45,3.8,5.4,1.44,3.6,6.5,NaN,NaN,NaN,1.40,3.75,7.0,1.4,4.0,7.50,1.45,4.0,6.0,1.47,4.0,6.25,1.44,4.0,6.5


In [287]:
a, b = leaguesDF.shape, matches.shape
print('Modified "leagueDF" shape: ' + str(a))
print('Original "matches" shape:  ' + str(b))

Modified "leagueDF" shape: (25979, 118)
Original "matches" shape:  (25979, 115)


In [288]:
# capture only two columns
temp = teamsDF[['team_api_id', 'team_long_name']]
# rename the column to match other df for key
temp = temp.rename(columns={'team_api_id':'home_team_api_id'}) 
# make sure there's no duplicates
temp.drop_duplicates(subset=['home_team_api_id', 'team_long_name'], inplace = True)
# merge them together on 'home_team_api_id'
leaguesDF = leaguesDF.merge(temp, on=['home_team_api_id'], how='left')
# rename again for away columns
temp = temp.rename(columns={'home_team_api_id':'away_team_api_id'}) 
# merge again
leaguesDF = leaguesDF.merge(temp, on=['away_team_api_id'], how='left')
# drop some useless features
leaguesDF.drop(['id_x', 'id_y', 'country_id', 'league_id', 'stage'], axis=1, inplace=True)
# create a copy of 'leagueDF' to simplify the information even more
leaguesFinal = leaguesDF
leaguesFinal = leaguesFinal[['Country', 'League', 'season', 'date', 'match_api_id', 'team_long_name_x', 
                       'team_long_name_y', 'home_team_goal', 'away_team_goal']]

In [289]:
leaguesFinal = leaguesFinal.rename(columns={'team_long_name_x':'Home Team', 'team_long_name_y':'Away Team'})
leaguesFinal.sample(3)

,Country,League,season,date,match_api_id,Home Team,Away Team,home_team_goal,away_team_goal
14144,Netherlands,Netherlands Eredivisie,2010/2011,2010-08-29 00:00:00,836170,De Graafschap,Ajax,0,5
19755,Scotland,Scotland Premier League,2008/2009,2008-12-20 00:00:00,490091,Kilmarnock,St. Mirren,0,1
19524,Portugal,Portugal Liga ZON Sagres,2015/2016,2016-02-22 00:00:00,2016009,Sporting CP,Boavista FC,2,0


In [290]:
def win(leaguesFinal):
    if leaguesFinal['home_team_goal'] > leaguesFinal['away_team_goal']:
        return leaguesFinal['Home Team']
    elif leaguesFinal['away_team_goal'] > leaguesFinal['home_team_goal']:
        return leaguesFinal['Away Team']
    elif leaguesFinal['home_team_goal'] == leaguesFinal['away_team_goal']:
        return "DRAW"
    
def loss(leaguesFinal):
    if leaguesFinal['home_team_goal'] < leaguesFinal['away_team_goal']:
        return leaguesFinal['Home Team']
    elif leaguesFinal['away_team_goal'] < leaguesFinal['home_team_goal']:
        return leaguesFinal['Away Team']
leaguesFinal["win"] = leaguesFinal.apply(lambda leaguesFinal:win(leaguesFinal),axis=1)
leaguesFinal["loss"] = leaguesFinal.apply(lambda leaguesFinal:loss(leaguesFinal),axis=1)

In [291]:
leaguesFinal.shape

(25979, 11)

In [292]:
leaguesFinal.sample(3)

,Country,League,season,date,match_api_id,Home Team,Away Team,home_team_goal,away_team_goal,win,loss
6368,France,France Ligue 1,2012/2013,2012-12-12 00:00:00,1216971,AC Ajaccio,Stade de Reims,2,0,AC Ajaccio,Stade de Reims
11934,Italy,Italy Serie A,2012/2013,2013-02-24 00:00:00,1287217,Parma,Catania,1,2,Catania,Parma
25278,Switzerland,Switzerland Super League,2012/2013,2012-10-07 00:00:00,1227796,FC Zürich,BSC Young Boys,1,1,DRAW,None


## Feature Engineering

### Team Record (Win/Loss) DataFrame

In [293]:
seasons = leaguesFinal['season'].unique()
teams = teamsDF['team_long_name'].unique()
df = []

for i in seasons:
    season = leaguesFinal['season'] == i
    season = leaguesFinal[season]
    for j in teams:
        team_season_wins = season['win'] == j
        team_season_win_record = team_season_wins[team_season_wins].count()
        team_season_loss = season['loss'] == j
        team_season_loss_record = team_season_loss[team_season_loss].count()
        df.append((j, i, team_season_win_record, team_season_loss_record))
df = pd.DataFrame(df, columns=('Team', 'Seasons', 'Wins', 'Losses'))
df = df.rename(columns={'Team':'Home Team'}) 
df2 = leaguesFinal[['Home Team', 'League']]
df2.drop_duplicates(subset = ['Home Team'],inplace = True)
df = df.merge(df2, on = 'Home Team')
df = df.rename(columns={'Home Team':'Team'}) 
teamRecords = df
teamRecords = teamRecords[['League', 'Team', 'Seasons', 'Wins', 'Losses']]
teamRecords = teamRecords[teamRecords.Wins != 0]
teamRecords = teamRecords[teamRecords.Losses != 0]
teamRecords.sample(5)

,League,Team,Seasons,Wins,Losses
1884,Scotland Premier League,Dundee FC,2012/2013,7,22
11,Belgium Jupiler League,Beerschot AC,2011/2012,9,12
845,Germany 1. Bundesliga,Borussia Mönchengladbach,2013/2014,16,11
1622,Portugal Liga ZON Sagres,Vitória Setúbal,2014/2015,7,19
1004,Italy Serie A,Catania,2012/2013,15,12


In [294]:
teamRecords.shape

(1453, 5)

### Team Goals by Season

In [295]:
df = []
home_goals = leaguesFinal.groupby(('Home Team', 'season'))['home_team_goal'].sum()
away_goals = leaguesFinal.groupby(('Away Team', 'season'))['away_team_goal'].sum()
a = home_goals.rename_axis(['Team','season'])
b = away_goals.rename_axis(['Team','season'])
df = (a.add(b, fill_value=0)).reset_index(name='Goals')
df = df.rename(columns={'season':'Seasons'}) 
teamRecords = teamRecords.merge(df, on = ['Team', 'Seasons'], how = 'left')
teamRecords.sort_values(['League', 'Team', 'Seasons'], ascending = True, inplace = True)
teamRecords.shape

(1453, 6)

In [296]:
teamRecords.sample(3)

,League,Team,Seasons,Wins,Losses,Goals
730,Italy Serie A,Sassuolo,2015/2016,16,9,49
136,England Premier League,West Bromwich Albion,2010/2011,12,15,56
1222,Spain LIGA BBVA,RCD Mallorca,2008/2009,14,15,53


### League Winners

In [297]:
# create a df of the team with the best record from each 'League' for each 'Seasons'
leagueWinners_season = teamRecords
leagueWinners_season.sort_values(['League', 'Seasons', 'Wins'], ascending = False, inplace = True)
leagueWinners_season = leagueWinners_season[leagueWinners_season.Wins > 10]
leagueWinners_season = leagueWinners_season.groupby(['League', 'Seasons']).first()
leagueWinners_season.sample(3)

,,Team,Wins,Losses,Goals
League,Seasons,,,,
Spain LIGA BBVA,2012/2013,FC Barcelona,32,2,115
Belgium Jupiler League,2012/2013,RSC Anderlecht,20,3,69
Italy Serie A,2010/2011,Milan,24,4,65


### Team Attributes Catagories

In [298]:
teamsDF.sample()

,team_api_id,team_fifa_api_id,team_long_name,team_short_name,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,chanceCreationCrossing,chanceCreationCrossingClass,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
170,8549,12.0,Middlesbrough,MID,2011-02-22 00:00:00,66,Balanced,NaN,Little,51,Mixed,Organised,64,Normal,64,Normal,48,Normal,Organised,54,Medium,44,Press,49,Normal,Cover


In [299]:
print(team_attributes.shape)
print(teamsDF.shape)
#print(teamAttributes.shape)

(1458, 25)
(1450, 26)


In [300]:
#teamsDF['date'] = pd.to_datetime(teamsDF['date'], infer_datetime_format = True)
#teamsDF['date'].astype(str)
#teamsDF['date'] = date.strftime('%Y-%m-%d')
teamsDF['date'].sample()

91    2013-09-20 00:00:00
Name: date, dtype: object

In [301]:
teamsDF['date'].dtype
#sensorData['datetime'] = pd.to_datetime(sensorData['datetime'], infer_datetime_format = True)

dtype('O')

In [ ]:
# datetime doesn't work with strptime in order to find the season from the date by month comparison
# need to split the 'date' string to lose the 00:00:00 first, then split the month and compare to create the season
lhs, rhs = "2.7.0_bf4fda703454".split("_", 1)

In [302]:
a = teamsDF['date'][0]
a
datee = datetime.datetime.strptime(a, "%Y-%m-%d")
#datee.month

ValueError: unconverted data remains:  00:00:00

teamAttributes = teamAttributes.rename(columns={'team__long_name':'Team'})  
# need to convert 'date' in teamAttributes to 'season' format
teamAttributes = teamRecords.merge(teamsDF, on = ['Team', 'Seasons'], how = 'left')

teamAttributes_quantities = pd.concat([teamsDF, pd.get_dummies(teamsDF.buildUpPlayDribblingClass)], 1).groupby(['team_long_name', 'date']).sum().reset_index()
teamAttributes_quantities.sample(3)

## Visualization

In [ ]:
goals_per_year = []
seasons = leaguesFinal['season'].unique()

for i in range(0,8):
    mask = leaguesFinal['season'] == seasons[i]
    goals = leaguesFinal[mask]['home_team_goal'].sum() + leaguesFinal[mask]['away_team_goal'].sum()
    goals_per_year.append(goals)

df = pd.DataFrame([goals_per_year]).transpose()
df['Season'] = ['2008/2009', '2009/2010', '2010/2011', '2011/2012', '2012/2013', '2013/2014', '2014/2015', '2015/2016']
df = df.rename(columns = {0 : 'Goals'})

#
sns.set_style("darkgrid")
plt.figure(figsize=(16, 4))
plt.plot(df['Season'], df['Goals'], color = 'royalblue')
plt.ylabel('Goals', fontsize = 14)
plt.xlabel('Season', fontsize = 14)
plt.title('Goals Scored in All Leagues by Year', fontsize = 16)
mpl.rcParams['agg.path.chunksize'] = 10000

In [ ]:
d = teamRecords.sort_values(by = 'Wins', ascending=False)
plt.figure(figsize=(14,5))
sns.barplot('Team', 'Wins', data = d[:50], color = 'b', label = 'Wins')
sns.barplot('Team', 'Losses', data = d[:50], color = 'r', label = 'Losses')
plt.xticks(rotation = 70, fontsize = 12)
plt.xlabel('Teams', fontsize = 14)
plt.ylabel('Games', fontsize = 14)
plt.legend(loc="best")
plt.title('Top Records by Team', fontsize = 16)
plt.show()

In [ ]:
sns.jointplot(leagueWinners_season['Wins'], leagueWinners_season['Goals'], kind = 'reg', color = 'b')
plt.title('Season Winner\'s Goal and Win Regression', y = 1.25, fontsize = 18)
plt.xlabel('Wins', fontsize=14)
plt.ylabel('Goals', fontsize=14)
plt.show()

In [ ]:
leagueLosers_season = teamRecords
leagueLosers_season.sort_values(['League', 'Seasons', 'Losses'], ascending = False, inplace = True)
leagueLosers_season = leagueLosers_season[leagueLosers_season.Losses > 10]
leagueLosers_season = leagueLosers_season.groupby(['League', 'Seasons']).first()

In [ ]:
leagueWinners_season['Side'] = 'Winners'
leagueLosers_season['Side'] = 'Losers'
scatter = leagueWinners_season.merge(leagueLosers_season, how = 'outer')
scatter = scatter[['Wins', 'Goals', 'Side']]
plt.figure(figsize=(7,7))
plt.title('Goals to Win Ratio for Highest Winning and Losing Teams', fontsize = 16)
plt.xlabel('Wins', fontsize=14)
plt.ylabel('Goals', fontsize=14)
ax = sns.scatterplot(x = 'Wins', y = 'Goals', hue = 'Side', data = scatter)

Graph team attributes - winning vs losing? winning in top of each league? 
histogram 
Using something like this to break apart attributes features and quantify them?...

#This first concatenates your dataframe and the output of the get_dummies call, then it 
#groups the result according to the relevant columns, takes the sum of those columns among 
#those groups and then resets the index so you don't have to deal with a multi-index data frame
errorFeatures = pd.concat([errorData, pd.get_dummies(errorData.errorID)], 1).groupby(['machineID', 'datetime']).sum().reset_index()